In [22]:
import requests
import json
import os

In [2]:
# Lista polskich nazw
spec_name_pl = [
    "pierwiosnek",
    "dzięcioł zielony",
    "synogarlica",
    "zięba",
    "kopciuszek",
    "wrona siwa",
    "bocian biały",
    "wróbel",
    "sikorka modra",
    "skowronek",
    "jaskółka",
    "jerzyk",
    "słowik",
    "jemiołuszka",
    "kos"
]

# Lista łacińskich nazw
spec_name = [
    "Phylloscopus collybita",
    "Picus viridis",
    "Streptopelia decaocto",
    "Fringilla coelebs",
    "Phoenicurus ochruros",
    "Corvus cornix",
    "Ciconia ciconia",
    "Passer domesticus",
    "Cyanistes caeruleus",
    "Alauda arvensis",
    "Hirundo rustica",
    "Apus apus",
    "Luscinia megarhynchos",
    "Bombycilla garrulus",
    "Turdus merula"
]



In [25]:
with open("spec_name_pl.json", "w") as file:
    json.dump(spec_name_pl, file)

with open("spec_name.json", "w") as file:
    json.dump(spec_name, file)

In [4]:
base_url = "https://www.xeno-canto.org/api/2/recordings"
len_gt = 15
q = "A"

In [5]:
for name in spec_name:
    page = 1
    recordings = []
    # name = spec_name[i]
    temp = name.split()
    while True:
        params = {  
            "query": f"{name} len_gt:{len_gt}",
            "page": page
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            recordings.extend(data['recordings'])
            num_pages = data['numPages']
            page += 1
            if page > num_pages:
                print(f"{name}, {data['numPages']}, {page-1}, {data['numRecordings']}, {len(recordings)}")
                break
        else:
            print(f"Failed to retrieve data: {name}, from page {page}", response.status_code)      
    with open(f'data_{temp[0]}-{temp[1]}.json', 'w') as f:
        json.dump(recordings, f)

Phylloscopus collybita, 13, 13, 6028, 6028
Picus viridis, 1, 1, 437, 437
Streptopelia decaocto, 2, 2, 655, 655
Fringilla coelebs, 13, 13, 6382, 6382
Phoenicurus ochruros, 3, 3, 1157, 1157
Corvus cornix, 3, 3, 1163, 1163
Ciconia ciconia, 1, 1, 94, 94
Passer domesticus, 5, 5, 2340, 2340
Cyanistes caeruleus, 8, 8, 3994, 3994
Alauda arvensis, 5, 5, 2410, 2410
Hirundo rustica, 3, 3, 1369, 1369
Apus apus, 1, 1, 435, 435
Luscinia megarhynchos, 5, 5, 2488, 2488
Bombycilla garrulus, 1, 1, 271, 271
Turdus merula, 13, 13, 6024, 6024


In [7]:
for specie in spec_name:
    temp = specie.split()
    with open (f'data/jsons/data_{temp[0]}-{temp[1]}.json', 'r') as f:
        data = json.load(f)
    for dict in data:
        dict['also'] = list(filter(lambda x: len(x) == 0, dict['also']))
    with open (f'data/jsons/data_{temp[0]}-{temp[1]}.json', 'w') as g:
        json.dump(data, g)
    

In [9]:
#sprawdzenie, czy usunieto wszystkie rekordy z niepustymi listami w 'also'
for specie in spec_name:
    temp = specie.split()
    with open (f'data/jsons/data_{temp[0]}-{temp[1]}.json', 'r') as f:
        data = json.load(f)
    d = 0
    for dict in data:
        if not dict['also']:
            d +=1
    print(specie, d, len(data))

Phylloscopus collybita 6028 6028
Picus viridis 437 437
Streptopelia decaocto 655 655
Fringilla coelebs 6382 6382
Phoenicurus ochruros 1157 1157
Corvus cornix 1163 1163
Ciconia ciconia 94 94
Passer domesticus 2340 2340
Cyanistes caeruleus 3994 3994
Alauda arvensis 2410 2410
Hirundo rustica 1369 1369
Apus apus 435 435
Luscinia megarhynchos 2488 2488
Bombycilla garrulus 271 271
Turdus merula 6024 6024


In [15]:
#foldery na nagrania
for specie in spec_name:
    temp = specie.split()
    folder = os.path.join('data/recordings/unmodified', f'{temp[0]}-{temp[1]}')
    os.makedirs(folder, exist_ok=True)

In [21]:
#pobieranie nagran
max_files = 300
for specie in spec_name:
    temp = specie.split()
    with open (f'data/jsons/data_{temp[0]}-{temp[1]}.json', 'r') as f:
        data = json.load(f)
    count = 0
    for dict in data:
        if count >= max_files:
            break
        link = dict['file']
        filename = link.split("/")[-2] + ".mp3"
        filepath = os.path.join(f'data/recordings/unmodified/{temp[0]}-{temp[1]}', filename)  # Pełna ścieżka pliku

        # Pobierz plik
        try:
            response = requests.get(link, stream=True)
            response.raise_for_status()  # Sprawdzenie, czy nie było błędów podczas pobierania
            with open(filepath, 'wb') as output_file:
                for chunk in response.iter_content(chunk_size=8192):
                    output_file.write(chunk)
            count += 1
            # print(f"Pobrano: {filepath}")
        except requests.RequestException as e:
            print(f"Nie udało się pobrać {link}: {e}")

Nie udało się pobrać https://xeno-canto.org/907043/download: 504 Server Error: Gateway Timeout for url: https://xeno-canto.org/907043/download
Nie udało się pobrać https://xeno-canto.org/914256/download: 404 Client Error: Not Found for url: https://xeno-canto.org/914256/download
Nie udało się pobrać https://xeno-canto.org/836979/download: 504 Server Error: Gateway Timeout for url: https://xeno-canto.org/836979/download
